In [1]:
# Import packages
from math import exp, log, pi, sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import fsolve
from __future__ import division
from numpy import inf
from math import *

In [2]:
# Define the parts for BS formula
def d1(Spot,K,T,rd,rf,sigma):
    return (np.log((Spot*exp((rd-rf)*T))/K)+0.5*sigma**2*T)/(sigma*sqrt(T))
           
def d2(Spot,K,T,rd,rf,sigma):
    return d1(Spot,K,T,rd,rf,sigma) - sigma * sqrt(T)

def bs_call(Spot,K,T,rd,rf,sigma):
    return exp(-(rd-rf) * T) * (Spot * exp((rd-rf) * T) * norm.cdf(d1(Spot,K,T,rd,rf,sigma))-K * norm.cdf(d2(Spot,K,T,rd,rf,sigma)))

In [3]:
# Calculate greeks for call and put
def call_delta(Spot,K,T,rd,rf,sigma):
    return exp(-rf * T)*norm.cdf(d1(Spot,K,T,rd,rf,sigma))
def call_gamma(Spot,K,T,rd,rf,sigma):
    return exp(-rf*T)*norm.pdf(d1(Spot,K,T,rd,rf,sigma))/(Spot*sigma*sqrt(T))
def call_vega(Spot,K,T,rd,rf,sigma):
    return 0.01*(S*norm.pdf(d1(Spot,K,T,rd,rf,sigma))*sqrt(T))
def call_theta(Spot,K,T,rd,rf,sigma):
    return 0.01*(-(Spot*norm.pdf(d1(Spot,K,T,rd,rf,sigma))*sigma)/(2*sqrt(T)) - rd*K*exp(-r*T)*norm.cdf(d2(Spot,K,T,rd,rf,sigma)))
def call_rho(Spot,K,T,rd,rf,sigma):
    return 0.01*(K*T*exp(-rd*T)*norm.cdf(d2(Spot,K,T,rd,rf,sigma)))
    
def put_delta(Spot,K,T,rd,rf,sigma):
    return -norm.cdf(-d1(Spot,K,T,rd,rf,sigma))
def put_gamma(Spot,K,T,rd,rf,sigma):
    return norm.pdf(d1(Spot,K,T,rd,rf,sigma))/(Spot*sigma*sqrt(T))
def put_vega(Spot,K,T,rd,rf,sigma):
    return 0.01*(Spot*norm.pdf(d1(Spot,K,T,rd,rf,sigma))*sqrt(T))
def put_theta(Spot,K,T,rd,rf,sigma):
    return 0.01*(-(Spot*norm.pdf(d1(Spot,K,T,rd,rf,sigma))*sigma)/(2*sqrt(T)) + rd*K*exp(-r*T)*norm.cdf(-d2(Spot,K,T,rd,rf,sigma)))
def put_rho(Spot,K,T,rd,rf,sigma):
    return 0.01*(-K*T*exp(-rd*T)*norm.cdf(-d2(Spot,K,T,rd,rf,sigma)))

In [4]:
# Variables
Spot = 1.17
rd = 0.03
rf = 0.012
Re = 0.06
T = 0.5
sigma = 0.095
K = 1.20873803
F = Spot* exp((rd-rf)*T)
Nom = 1000000
M = 1000

In [5]:
# Calculate the DCD premium with the formula on book
def dcd_premium(M, Nom, T, rd, Re):
    return (M-Nom*T*(rd-Re))/Nom

In [6]:
# Black & Scholes Solver: calculate the value of any missing variable in BS formula
def black(cp=None, Spot=None, k=None, t=None, rd=None, rf=None, v=None, price=None):
    """ General Purpose BSM machine. Leave one parameter blank and that is the parameter that will
        be solved for.
            cp   = "c" for a call, anything else assumes a put
            f    = Forward Price of the underlying asset
            k    = Strike Price
            t    = Time until maturity (in years)
            rd   = Domestic Interest rate
            rf   = Foreign interest rate
            v    = Implied volatility
            full = If True, function returns a dictionary with price and all sensitivities
                   Otherwise only the calculated/calibrated parameter will be returned
    """

    D1 = lambda Spot, k, v, t, rd, rf: (np.log((Spot*exp((rd-rf)*t))/k)+0.5*v**2*t)/(v*sqrt(t))
    D2 = lambda Spot, k, v, t, rd, rf: D1(Spot, k, v, t, rd, rf)-v*sqrt(t)

    optionType = 1 if cp.upper()[0] == "C" else -1   # If the first letter is not "c" or "C" it must be a put option
    parameters = locals().copy()  # The locals() method updates and returns a dictionary of the current local symbol table.

    def _black(cp, Spot, k, t, rd, rf, v, price, calibration=False, **kwargs):
        d1, d2 = D1(Spot, k, v, t, rd, rf), D2(Spot, k, v, t, rd, rf)
        price = optionType * exp(-(rd-rf) * t)*((Spot*exp((rd-rf)*t)*norm.cdf(optionType*d1)-k*norm.cdf(optionType*d2)))
        return price

    def _calibrate(value, field, parameters):
        parameters.update({field: value[0]})
        return abs(_black(calibration=True, **parameters) - price, ) #price is from the beginning or _black?

    missing = [a for a in ["Spot", "k", "t", "rd", "rf", "v", "price"] if parameters[a] is None]

    if len(missing) > 1:
        raise Exception("Too many missing variables from: %s " % missing)
    if len(missing) == 0:
        raise Exception("All variables assigned - nothing to solve")

    if missing[0] != "price":
        # if we are missing any parameter different from price we need to calibrate
        result = fsolve(_calibrate, 0.1, args=(missing[0], parameters))
        return missing[0],float(result)
    
            
    return _black(**parameters)

In [7]:
# Get delta hedge strategy and value of missing variable
def BS_delta_hedge(Spot, rd, rf, T, sigma, Nom, Re, M, **kwargs):
    K = black("c",v=sigma,Spot=Spot,t=T,rd=rd, rf = rf, price=dcd_premium(M, Nom, T, rd, Re))[1]
    delta = call_delta(Spot,K,T,rd,rf,sigma)
    delta_hedge = Nom * delta
    delta_hedge_opposite = Nom * delta * Spot
    return np.round(K,8), 'Delta is: %a, Bank should sell: %a EUR and buy %a USD.' %(np.round(delta,6),np.round(delta_hedge,2), np.round(delta_hedge_opposite,2))

In [8]:
# DCD Solver - to get sales margin and enhanced rate
def DCD_premium(M=None, Nom=None, t=None, rd=None, Re=None, price=None):
    """ General Purpose BSM machine. Leave one parameter blank and that is the parameter that will
        be solved for.
            M    = Sales Margin
            t    = Time to maturity (in years)
            rd   = Domestic Interest rate
            re   = Enhanced rate
    """
    parameters = locals().copy()
    
    def _DCD(M, Nom, t, rd, Re, price, calibration=False, **kwargs):
        price = (M-Nom*t*(rd-Re))/Nom
        return price        

    def _calibrate(value, field, parameters):
        parameters.update({field: value[0]})
        return abs(_DCD(calibration=True, **parameters) - price, )

    missing = [d for d in ["M", "Nom", "t", "rd", "Re", "price"] if parameters[d] is None]

    if len(missing) > 1:
        raise Exception("Too many missing variables from: %s " % missing)
    if len(missing) == 0:
        raise Exception("All variables assigned - nothing to solve")

    if missing[0] != "price":
        # if we are missing any parameter different from price we need to calibrate
        result = fsolve(_calibrate, 0.1, args=(missing[0], parameters))
        return missing[0], float(result)  # If full=False we simply return the calibrated parameter

    return _DCD(**parameters)

In [9]:
# Get delta hedge strategy and sales margin
def dcd_delta_hedge_M(Spot, k, rd, rf, t, sigma, Nom, Re, price, **kwargs):
    Sales_Margin = DCD_premium(Nom=Nom, t=t, rd=rd, Re=Re, price=price)[1]
    delta = call_delta(Spot,k,t,rd,rf,sigma)
    delta_hedge = Nom * delta
    delta_hedge_opposite = Nom * delta * Spot
    return np.round(Sales_Margin,4),'Delta is: %a, Bank should sell: %a EUR and buy %a USD.' %(np.round(delta,6), np.round(delta_hedge,2), np.round(delta_hedge_opposite,2))

In [10]:
# Get delta hedge strategy and enhanced rate
def dcd_delta_hedge_E(Spot, k, rd, rf, t, sigma, Nom, M, price, **kwargs):
    Enhanced_rate = DCD_premium(M=M, Nom=Nom, t=t, rd=rd, price=price)[1]
    delta = call_delta(Spot,k,t,rd,rf,sigma)
    delta_hedge = Nom * delta
    delta_hedge_opposite = Nom * delta * Spot
    return np.round(Enhanced_rate,6), 'Delta is: %a, Bank should sell: %a EUR and buy %a USD.' %(np.round(delta,6), np.round(delta_hedge,2), np.round(delta_hedge_opposite,2))

In [11]:
# Final DCD Pricer - combine the three functions above
def FX_Pricing_Tool(Spot=None, k=None, t=None, rd=None, rf=None, v=None, M=None, Nom=None, Re=None, price=None, **kwargs):
    #optionType = 1 if cp.upper()[0] == "C" else -1  
    parameters = locals().copy()
    missing = [a for a in ["Spot", "k", "t", "rd", "rf", "v", "M", "Nom", "Re", "price"] if parameters[a] is None]
    if len(missing) > 1:
        raise Exception("Too many missing variables from: %s " % missing)
    if len(missing) == 0:
        raise Exception("All variables assigned - nothing to solve")

    if missing[0] == "k":
        return BS_delta_hedge(Spot, rd, rf, t, v, Nom, Re, M)
    elif missing[0] == "M":
        return dcd_delta_hedge_M(Spot, k, rd, rf, t, v, Nom, Re, price)
    elif missing[0] == "Re":
        return dcd_delta_hedge_E(Spot, k, rd, rf, t, v, Nom, M, price)

In [12]:
# Pricing tool GUI building
from tkinter import *
from tkinter import messagebox
import scipy.stats as sps

In [13]:
root = Tk()
root.geometry("500x650")

root.wm_title('DCD price Calculator')
Label(root, text = 'Please input relevant parameters, '
      'then click "Calculate" button.').grid(row = 0, column = 0, columnspan = 3)

cp = StringVar()
Label(root, text='Option Type').grid(row=2, column=0, sticky=W)
Radiobutton(root, text='Call', variable=cp, value='c').grid(row=2, column=1)
Radiobutton(root, text='Put', variable=cp, value='p').grid(row=2, column=2)

In [14]:
global label_list 
label_list = []
global elist
elist = []
plist = ['Margin', 'Enhanced Rate', 'Notional', 'Time to Maturity', 'Rd', 
         'Spot', 'k', 'Rf', 'Volatility', "Options price"]

# Build a derop down list to choose which variable do we want to know
def choose_scenarios(*args):    
    global given_var
    given_var = [x for x in plist if x != args[0]]
    global missing
    missing = args[0]
        
    for widget_l in label_list:
        widget_l.destroy()
    #for widget_e in elist:
        #widget_e.destroy()
    r = 4
    elist.clear()
    label_list.clear()
    for param in given_var:
        label = Label(root, text=param)
        label.grid(row=r, column=0, sticky=W)
        label_list.append(label)
        e = Entry(root)
        e.grid(row=r, column=1, columnspan=2, sticky=W+E)
        elist.append(e)
        r += 1
        
options = [ 
    "k", 
    "Margin", 
    "Enhanced Rate"
] 

clicked = StringVar(root)   
# initial menu text 
clicked.set("What do you want to know") 

# Create Dropdown menu 
drop = OptionMenu(root, clicked, *options, command = choose_scenarios) 
drop.grid() 

In [15]:
# Step 1: calculate option price with another avaliable formula (either BS formula or DCD formula)
r=13
answ = Label(root, text='The options price is as follows:')
answ.grid(row=r, columnspan=3)
r += 1

p = Label(root)
p.grid(row=r, columnspan=2, sticky=E)
Label(root, text='Calculate price first: ').grid(row=r, sticky=W)

def price_calc():
    if "k" not in given_var:
        vlist = []
        for e in elist:
            try:
                j = float(e.get())
                vlist.append(j)
            except:
                answ.config(text='Invalid Input(s). Please input correct parameter(s)', fg='red')
                e.delete(0, len(e.get()))
                return 0
        o_price = dcd_premium(vlist[0], vlist[2], vlist[3], vlist[4], vlist[1])
        answ.config(text='option price is:', fg='black')
        p.config(text=str("%.8f"%o_price))
    
    else:
        vlist = []
        for e in elist:
            try:
                j = float(e.get())
                vlist.append(j)
            except:
                answ.config(text='Invalid Input(s). Please input correct parameter(s)', fg='red')
                e.delete(0, len(e.get()))
                return 0

        o = bs_call(vlist[4], vlist[5], vlist[2], vlist[3], vlist[6], vlist[7])
        answ.config(text='option price is:', fg='black')
        p.config(text=str("%.8f"%o))
        
Button(root, text='Options price', command=price_calc).grid(row=r, column = 2)
r += 2

In [16]:
# Space for answer
answ_2 = Label(root, text='The result is as follows:')
answ_2.grid(row=r, columnspan=3)
r += 1

outcome = Label(root)
outcome.grid(row=r, columnspan=2, sticky=E)
r += 1

outcome2 = Label(root)
outcome2.grid(row=r, columnspan=3, sticky=E)

In [17]:
# Calculate the variable that we want to know (K, Margin, or Enhanced Rate)
def final_calc():
    if "k" not in given_var:
        vlist = []
        for e in elist:
            i = float(e.get())
            vlist.append(i)
    
        o_final = FX_Pricing_Tool(M=vlist[0], Re=vlist[1], Nom=vlist[2], t=vlist[3], rd=vlist[4], Spot=vlist[5], rf=vlist[6], v=vlist[7], price=vlist[8])
        answ_2.config(text='The result is as follows:', fg='black')
        #outcome.config(text=str(o_final))
        outcome.config(text=o_final[0])
        outcome2.config(text=o_final[1])
        
    elif "Margin" not in given_var:
        vlist = []
        for e in elist:
            i = float(e.get())
            vlist.append(i)

        o_final = FX_Pricing_Tool(k=vlist[5], Re=vlist[0], Nom=vlist[1], t=vlist[2], rd=vlist[3], Spot=vlist[4], rf=vlist[6], v=vlist[7], price=vlist[8])
        answ_2.config(text='The result is as follows:', fg='black')
        #outcome.config(text=str(o_final))
        if o_final[0] < 0:
            outcome.config(text=o_final[0], fg = 'red')
            messagebox.showwarning("Invalid Output", "Your margin cannot be negative! Please change input!")
        else:
            outcome.config(text=o_final[0])
            outcome2.config(text=o_final[1])

    elif "Enhanced Rate" not in given_var:
        vlist = []
        for e in elist:
            i = float(e.get())
            vlist.append(i)

        o_final = FX_Pricing_Tool(k=vlist[5], M=vlist[0], Nom=vlist[1], t=vlist[2], rd=vlist[3], Spot=vlist[4], rf=vlist[6], v=vlist[7], price=vlist[8])
        answ_2.config(text='The result is as follows:', fg='black')
        
        if o_final[0] < vlist[3]:
            outcome.config(text=o_final[0], fg = 'red')
            messagebox.showwarning("Invalid Output", "Enhanced rate cannot be lower than market rate!")
        else:
            outcome.config(text=o_final[0])
            outcome2.config(text=o_final[1])

Button(root, text='Calculate', command=final_calc).grid(row=r-1, column = 2)
root.mainloop()